ANALISIS DE CONTINGENCIAS CON PYTHON

Funcion para la matriz B y B''

In [103]:
import numpy as np
import pandas as pd

In [104]:
def calculate_B_matrix(linedata, Sbase, Vbase):
    j = 1j
    nbr = int(len(linedata[:, 0]))
    nbus = int(max(max(linedata[:, 0]
                       ), max(linedata[:, 1])))
    B_matrix = np.zeros((nbus, nbus), dtype=np.float64) 

    # Paso 1: Calcular Zbase, pero al tener los datos en pu, Zbase = 1
    Zbase = 1

    # Pase 2: Armar la matriz B
    B_values = Zbase / linedata[:, 3]

    for k in range(nbr):
        nl = int(linedata[k, 0])
        nr = int(linedata[k, 1])

        # Paso 3: Calcular los elementos de la matriz B
        B_matrix[nl - 1, nl - 1] += B_values[k]
        B_matrix[nr - 1, nr - 1] += B_values[k]
        B_matrix[nl - 1, nr - 1] -= B_values[k]
        B_matrix[nr - 1, nl - 1] -= B_values[k]
    # Remueve la fila y columna de la barra slack
    B_matrix_result = B_matrix[1:, 1:]
    return B_matrix, B_matrix_result


In [105]:
# Sistema de prueba IEEE 14 bus
# linedata = [De_Bus  A_Bus  R  X  B  TR]
linedata = np.array([
    [1, 2, 0.01938, 0.05917, 0.0528, 1],
    [1, 5, 0.05403, 0.22304, 0.0492, 1],
    [2, 3, 0.04699, 0.19797, 0.0438, 1],
    [2, 4, 0.05811, 0.17632, 0.0374, 1],
    [2, 5, 0.05695, 0.17388, 0.034, 1],
    [3, 4, 0.06701, 0.17103, 0.0346, 1],
    [4, 5, 0.01335, 0.04211, 0.0128, 1],
    [4, 7, 0.0, 0.20912, 0.0, 1],
    [4, 9, 0.0, 0.55618, 0.0, 1],
    [5, 6, 0.0, 0.25202, 0.0, 1],
    [6, 11, 0.09498, 0.1989, 0.0, 1],
    [6, 12, 0.12291, 0.25581, 0.0, 1],
    [6, 13, 0.06615, 0.13027, 0.0, 1],
    [7, 8, 0.0, 0.17615, 0.0, 1],
    [7, 9, 0.0, 0.11001, 0.0, 1],
    [9, 10, 0.03181, 0.0845, 0.0, 1],
    [9, 14, 0.12711, 0.27038, 0.0, 1],
    [10, 11, 0.08205, 0.19207, 0.0, 1],
    [12, 13, 0.22092, 0.19988, 0.0, 1],
    [13, 14, 0.17093, 0.34802, 0.0, 1]
])
Sbase = 100  # Potencia base del sistema en MVA
Vbase = 60  # Voltaje base del sistema en kV (Lado de alta tensión)
#Potencias en las barras, + es generada, - es consumida
#power_injected = [De_Bus  MW]
potencia_barras = np.array([
    [2, 42.4-21.7],
    [3, 23.4-94.2],
    [4, -47.8],
    [5, -7.6],
    [6, 12.2-11.2],
    [7, 0],
    [8, 17.4],
    [9, -29.5],
    [10, -9],
    [11, -3.5],
    [12, -6.1],
    [13, -13.5],
    [14, -14.9]
])
Zbase = (Vbase ** 2) / Sbase

In [106]:
B_matrix, B_matrix_result = calculate_B_matrix(linedata, Sbase, Vbase)
# Display B matrix result as a table
B_table_result = pd.DataFrame(B_matrix_result, index=range(2, B_matrix_result.shape[0] + 2),
                              columns=range(2, B_matrix_result.shape[1] + 2))

print("B Matrix Result:")
print(B_table_result)

B Matrix Result:
           2          3          4          5          6          7        8    
2   33.374326  -5.051270  -5.671506  -5.751093   0.000000   0.000000  0.00000  \
3   -5.051270  10.898198  -5.846927   0.000000   0.000000   0.000000  0.00000   
4   -5.671506  -5.846927  41.845685 -23.747328   0.000000  -4.781943  0.00000   
5   -5.751093   0.000000 -23.747328  37.949861  -3.967939   0.000000  0.00000   
6    0.000000   0.000000   0.000000  -3.967939  20.581107   0.000000  0.00000   
7    0.000000   0.000000  -4.781943   0.000000   0.000000  19.549006 -5.67698   
8    0.000000   0.000000   0.000000   0.000000   0.000000  -5.676980  5.67698   
9    0.000000   0.000000  -1.797979   0.000000   0.000000  -9.090083  0.00000   
10   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000  0.00000   
11   0.000000   0.000000   0.000000   0.000000  -5.027652   0.000000  0.00000   
12   0.000000   0.000000   0.000000   0.000000  -3.909151   0.000000  0.00000   
13   0.0000

In [107]:
#Codigo para calcular el flujo DC
Theta = np.linalg.inv(B_matrix_result) @ potencia_barras[:,1:] / Sbase
Theta = np.vstack((np.array([[0]]),Theta))
print(Theta)

[[ 0.        ]
 [-0.06465996]
 [-0.17101996]
 [-0.14181712]
 [-0.12115918]
 [-0.19716299]
 [-0.1743726 ]
 [-0.1437225 ]
 [-0.21064052]
 [-0.21573595]
 [-0.21003164]
 [-0.21652628]
 [-0.21946332]
 [-0.23717037]]


In [108]:
dc_power_flows = []
for k in range(len(linedata)):
    nl = int(linedata[k, 0])  # De-bus
    nr = int(linedata[k, 1])  # A-bus
    
    # Obtenemos el valor de theta en los buses De-bus y A-bus
    theta_from = Theta[nl-1]
    theta_to = Theta[nr-1]
    # Se calcula el flujo DC para el elemento con la formula P = B * (theta_from - theta_to)
    B = B_matrix[nl - 1, nr - 1]
    P = -B * (theta_from - theta_to)
    dc_power_flows.append(P)

dc_power_flows = np.array(dc_power_flows)
print("\nFlujo de potencia DC para cada elemento en linedata:")
print(dc_power_flows)
np.shape(dc_power_flows)


Flujo de potencia DC para cada elemento en linedata:
[[ 1.09278275]
 [ 0.54321725]
 [ 0.53725313]
 [ 0.43759736]
 [ 0.32493226]
 [-0.17074687]
 [-0.490571  ]
 [ 0.15567845]
 [ 0.12374304]
 [ 0.30157851]
 [ 0.0646991 ]
 [ 0.07569401]
 [ 0.1711854 ]
 [-0.174     ]
 [ 0.32967845]
 [ 0.0603009 ]
 [ 0.09812059]
 [-0.0296991 ]
 [ 0.01469401]
 [ 0.05087941]]


(20, 1)

In [109]:
#Calculos para los coeficientes por salida de generador
#Calculo de F
F = modified_matrix = np.pad(np.linalg.inv(B_matrix_result), ((1, 0), (1, 0)), mode='constant')

In [110]:
#Insertar la informacion del BUS en el que se encuentra el generador a ser retirado
ni = 2

In [111]:
ali = []
for k in range(len(linedata)):
    nl = int(linedata[k, 0])  # De-bus
    nr = int(linedata[k, 1])  # A-bus

    ali.append(1/(1/-B_matrix[nl-1,nr-1])*(F[nl-1,ni-1]-F[nr-1,ni-1]))
ali = np.array(ali)[:, np.newaxis]
print("ali coefficients:")
print(ali)
print(np.shape(ali))

ali coefficients:
[[-8.38026301e-01]
 [-1.61973699e-01]
 [ 2.73434474e-02]
 [ 5.72239797e-02]
 [ 7.74062715e-02]
 [ 2.73434474e-02]
 [ 8.00206695e-02]
 [ 2.88905148e-03]
 [ 1.65770613e-03]
 [-4.54675761e-03]
 [-2.73795096e-03]
 [-4.02131958e-04]
 [-1.40667469e-03]
 [ 3.93919609e-17]
 [ 2.88905148e-03]
 [ 2.73795096e-03]
 [ 1.80880665e-03]
 [ 2.73795096e-03]
 [-4.02131958e-04]
 [-1.80880665e-03]]
(20, 1)


In [112]:
#Calculo del nuevo flujo DC con los factores de correccion tras la salida del generador
new_dc_power_flows = []
for k in range(len(linedata)):
    nl = int(linedata[k, 0])  
    nr = int(linedata[k, 1])  
    NP = dc_power_flows[k] + ali[k] * -potencia_barras[ni-2,1]/Sbase 
    new_dc_power_flows.append(NP)
new_dc_power_flows = np.array(new_dc_power_flows)
#Imprime los nuevos flujos DC con ni, correspondiente a la variable ni
print("\nNuevos flujo DC tras la salida del generador de la barra {}".format(ni))
print(new_dc_power_flows)


Nuevos flujo DC tras la salida del generador de la barra 2
[[ 1.26625419]
 [ 0.57674581]
 [ 0.53159304]
 [ 0.42575199]
 [ 0.30890916]
 [-0.17640696]
 [-0.50713528]
 [ 0.15508041]
 [ 0.12339989]
 [ 0.30251969]
 [ 0.06526586]
 [ 0.07577725]
 [ 0.17147659]
 [-0.174     ]
 [ 0.32908041]
 [ 0.05973414]
 [ 0.09774616]
 [-0.03026586]
 [ 0.01477725]
 [ 0.05125384]]


Analisis de contingencias por salida de linea

In [113]:
linedata = np.array([
    [1, 2, 0.01938, 0.05917, 0.0528, 1],
    [1, 5, 0.05403, 0.22304, 0.0492, 1],
    [2, 3, 0.04699, 0.19797, 0.0438, 1],
    [2, 4, 0.05811, 0.17632, 0.0374, 1],
    [2, 5, 0.05695, 0.17388, 0.034, 1],
    [3, 4, 0.06701, 0.17103, 0.0346, 1],
    [4, 5, 0.01335, 0.04211, 0.0128, 1],
    [4, 7, 0.0, 0.20912, 0.0, 1],
    [4, 9, 0.0, 0.55618, 0.0, 1],
    [5, 6, 0.0, 0.25202, 0.0, 1],
    [6, 11, 0.09498, 0.1989, 0.0, 1],
    [6, 12, 0.12291, 0.25581, 0.0, 1],
    [6, 13, 0.06615, 0.13027, 0.0, 1],
    [7, 8, 0.0, 0.17615, 0.0, 1],
    [7, 9, 0.0, 0.11001, 0.0, 1],
    [9, 10, 0.03181, 0.0845, 0.0, 1],
    [9, 14, 0.12711, 0.27038, 0.0, 1],
    [10, 11, 0.08205, 0.19207, 0.0, 1],
    [12, 13, 0.22092, 0.19988, 0.0, 1],
    [13, 14, 0.17093, 0.34802, 0.0, 1]
])
power_injected = np.array([
    [2, 42.4-21.7],
    [3, 23.4-94.2],
    [4, -47.8],
    [5, -7.6],
    [6, 12.2-11.2],
    [7, 0],
    [8, 17.4],
    [9, -29.5],
    [10, -9],
    [11, -3.5],
    [12, -6.1],
    [13, -13.5],
    [14, -14.9]
])
Sbase = 100  # System base MVA
Vbase = 60  # System base voltage kV

In [114]:
B_matrix, B_matrix_result= calculate_B_matrix(linedata, Sbase, Vbase)
B_matrix

array([[ 21.38395703, -16.90045631,   0.        ,   0.        ,
         -4.48350072,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-16.90045631,  33.37432577,  -5.05127039,  -5.67150635,
         -5.75109271,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,  -5.05127039,  10.89819783,  -5.84692744,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,  -5.67150635,  -5.84692744,  41.84568467,
        -23.74732843,   0.        ,  -4.78194338,   0.        ,
         -1.79797907,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ -4.48350072,  -5.75109271,   0.        , -23.74732843,
         37.9498609 

In [115]:
#Codigo para el flujo de potencia
Theta = np.linalg.inv(B_matrix_result) @ power_injected[:,1:] / Sbase
Theta = np.vstack((np.array([[0]]),Theta))
print(Theta)

[[ 0.        ]
 [-0.06465996]
 [-0.17101996]
 [-0.14181712]
 [-0.12115918]
 [-0.19716299]
 [-0.1743726 ]
 [-0.1437225 ]
 [-0.21064052]
 [-0.21573595]
 [-0.21003164]
 [-0.21652628]
 [-0.21946332]
 [-0.23717037]]


In [116]:
dc_power_flows = []
for k in range(len(linedata)):
    nl = int(linedata[k, 0]) 
    nr = int(linedata[k, 1])  
    
    theta_from = Theta[nl-1]
    theta_to = Theta[nr-1]
    B = B_matrix[nl - 1, nr - 1]
    P = -B * (theta_from - theta_to)
    dc_power_flows.append(P)

dc_power_flows = np.array(dc_power_flows)
print("\nFlujo de potencia en DC para cada elemento en linedata:")
print(dc_power_flows)



Flujo de potencia en DC para cada elemento en linedata:
[[ 1.09278275]
 [ 0.54321725]
 [ 0.53725313]
 [ 0.43759736]
 [ 0.32493226]
 [-0.17074687]
 [-0.490571  ]
 [ 0.15567845]
 [ 0.12374304]
 [ 0.30157851]
 [ 0.0646991 ]
 [ 0.07569401]
 [ 0.1711854 ]
 [-0.174     ]
 [ 0.32967845]
 [ 0.0603009 ]
 [ 0.09812059]
 [-0.0296991 ]
 [ 0.01469401]
 [ 0.05087941]]


#Factores por salida de lines dlk

In [117]:
#Calculo de F
F = modified_matrix = np.pad(np.linalg.inv(B_matrix_result), ((1, 0), (1, 0)), mode='constant')
print(F)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.04958602 0.04417283 0.03949628 0.03612661 0.03727249
  0.03889213 0.03889213 0.0385743  0.03834294 0.03781707 0.03737536
  0.03745574 0.03808524]
 [0.         0.04417283 0.14949934 0.06946288 0.05653148 0.06092887
  0.06714437 0.06714437 0.06592469 0.06503684 0.06301874 0.06132364
  0.0616321  0.06404786]
 [0.         0.03949628 0.06946288 0.09535159 0.07415963 0.08136607
  0.09155202 0.09155202 0.08955321 0.0880982  0.08479094 0.08201302
  0.08251852 0.08647746]
 [0.         0.03612661 0.05653148 0.07415963 0.08686153 0.08254219
  0.076437   0.076437   0.07763503 0.07850712 0.08048941 0.08215442
  0.08185144 0.07947855]
 [0.         0.03727249 0.06092887 0.08136607 0.08254219 0.24846154
  0.12676237 0.12676237 0.15064362 0.16802771 0.20754206 0.24073196
  0.23469237 0.18739184]
 [0.         0.03889213 0.06714437 0.09155202 0.07

In [118]:
dlk = []
for k in range(len(linedata)):
    ni = int(linedata[k, 0])  # De-bus
    nj = int(linedata[k, 1])  # A-bus
    nm = 0 #Salida de linea 1 [1-2]
    nn = 1 #Salida de linea 2
    dlk.append(1/(1/(-B_matrix[ni-1,nj-1]))*((1/(-B_matrix[nm,nn])*(F[ni-1,nm]-F[nj-1,nm]-F[ni-1,nn]+F[nj-1,nn]))/(1/(-B_matrix[nm,nn])-(F[nm,nm]+F[nn,nn]-2*F[nm,nn]))))
dlk = np.array(dlk)[:, np.newaxis]
print("Coeficientes dlk")
print(dlk)

Coeficientes dlk
[[ 5.17384185e+00]
 [ 1.00000000e+00]
 [-1.68814120e-01]
 [-3.53291801e-01]
 [-4.77894079e-01]
 [-1.68814120e-01]
 [-4.94034959e-01]
 [-1.78365469e-02]
 [-1.02344155e-02]
 [ 2.80709624e-02]
 [ 1.69036762e-02]
 [ 2.48269911e-03]
 [ 8.68458708e-03]
 [-2.43199737e-16]
 [-1.78365469e-02]
 [-1.69036762e-02]
 [-1.11672862e-02]
 [-1.69036762e-02]
 [ 2.48269911e-03]
 [ 1.11672862e-02]]


In [119]:
#Nuevo flujo DC tras la contingencia de la linea
new_dc_power_flows = []
for k in range(len(linedata)):
    nl = int(linedata[k, 0])  # From-bus
    nr = int(linedata[k, 1])  # To-bus
    NP = dc_power_flows[k] + dlk[k] * dc_power_flows[1] 
    new_dc_power_flows.append(NP)
new_dc_power_flows = np.array(new_dc_power_flows)
print("\nNuevo flujo DC tras la contingencia de la linea {}-{}:".format(nm,nn))
print(new_dc_power_flows)


Nuevo flujo DC tras la contingencia de la linea 0-1:
[[ 3.90330291]
 [ 1.08643451]
 [ 0.44555039]
 [ 0.24568315]
 [ 0.06533195]
 [-0.26244961]
 [-0.75893931]
 [ 0.14598933]
 [ 0.11818353]
 [ 0.31682714]
 [ 0.07388147]
 [ 0.07704265]
 [ 0.17590302]
 [-0.174     ]
 [ 0.31998933]
 [ 0.05111853]
 [ 0.09205433]
 [-0.03888147]
 [ 0.01604265]
 [ 0.05694567]]
